In [1]:
import calendar
import datetime as dt
import dateutil.parser as dp
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import os

from IPython.display import display, Markdown
from functools import partial
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

# Setting up key, token and Service Account

In [2]:
with open('keys.json') as file:
    keys = json.load(file)
    api_key = keys['trello']['api_key']
    token = keys['trello']['token']
    spreadsheet_key = keys['sheet']['spreadsheet_key']
    sa_file = keys['sheet']['sa_file']

# Extracting Sheet Data

In [3]:
def convert_sheet_date(sheet_date):
    conversion_table = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
        'May': '05',
        'June': '06',
        'July': '07',
        'August': '08',
        'September': '09',
        'October': '10',
        'November': '11',
        'December': '12'
    }
    
    month = conversion_table[sheet_date.split('-')[0].strip()]
    year = sheet_date.split('-')[1].strip()
    
    return year + '-' + month

In [4]:
def get_consolidated_sheet():
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(sa_file, scope)
    service = build('sheets', 'v4', credentials=credentials)

    SAMPLE_RANGE_NAME = 'Consolidated'
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_key,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    
    return values

In [5]:
def get_total_by_date():
    table = get_consolidated_sheet()
    
    header_row = table[0]
    columns = [value for index, value in enumerate(header_row) if value] 
    converted_columns = [ convert_sheet_date(x) for x in columns[1:]]
    
    total_row = [value for value in table if value and value[0] == 'Total'][0]
    # getting only the executed column
    total = [value for index, value in enumerate(total_row) if index % 3 == 0]

    total.pop(0)
    
    total_by_date = [ [converted_columns[index], value] for index, value in enumerate(total)]
    
    return total_by_date

In [6]:
def create_ftes_dataframe():
    # from deprecated sheet, it will never be changed
    old_data = [
        ['2020-03', 4.0],
        ['2020-04', 6.15],
        ['2020-05', 6.25],
        ['2020-06', 6.0],
        ['2020-07', 3.65],
        ['2020-08', 4.57],
        ['2020-09', 4.52],
        ['2020-10', 4.9],
        ['2020-11', 4.7]
    ]
    
    new_data = get_total_by_date()
    
    full_data = old_data + new_data
    
    total_fte = pd.DataFrame(full_data, columns=['month_base', 'fte'])
    
    total_fte['month_base'] = pd.to_datetime(total_fte['month_base'])
    total_fte['month'] = pd.PeriodIndex(total_fte['month_base'], freq='M')
    total_fte['quarter'] = pd.PeriodIndex(total_fte['month_base'], freq='Q')
    total_fte['fte'] = total_fte['fte'].astype(float)
    
    del total_fte['month_base']
    
    return total_fte

In [7]:
total_fte = create_ftes_dataframe()

In [8]:
total_fte

,fte,month,quarter
0,4.00,2020-03,2020Q1
1,6.15,2020-04,2020Q2
2,6.25,2020-05,2020Q2
3,6.00,2020-06,2020Q2
4,3.65,2020-07,2020Q3
5,4.57,2020-08,2020Q3
6,4.52,2020-09,2020Q3
7,4.90,2020-10,2020Q4
8,4.70,2020-11,2020Q4
9,4.90,2020-12,2020Q4


# Extracting Trello Data

In [9]:
def get_data_from_trello_api(url):
    headers = {
       "Accept": "application/json"
    }
    
    query = {
       'key': api_key,
       'token': token
    }
    
    response = requests.request("GET", url, headers=headers, params=query)
    
    if response.status_code > 299:
        raise Exception('Something went wrong with the request {0} '\
                        'with status: {1}'.format(url, response.status_code))
    
    return json.loads(response.text)

In [10]:
def read_json(json_name):
    with open(json_name) as file:
        json_opened = json.load(file)
           
    return json_opened 

In [11]:
def write_json(json_name, content_to_write):
    with open(json_name, 'w') as json_file:
        json.dump(content_to_write, json_file, indent=4, sort_keys=True)

In [12]:
def generate_timestamp():
    current_timestamp = dt.datetime.now().strftime('%d-%m-%Y')
    
    return current_timestamp

In [13]:
def create_folder_for_dumping(name, current_timestamp):
    if not os.path.exists('dumps'):
        os.mkdir('dumps')
    
    if not os.path.exists('dumps/' + name):
        os.mkdir('dumps/' + name)
    
    if not os.path.exists('dumps/' + name + '/' + current_timestamp):
        os.mkdir('dumps/' + name + '/' + current_timestamp)

In [14]:
def get_data_from_dump(board_name, dump_name, timestamp):
    path = 'dumps/' + board_name + '/' + timestamp + '/dump_' + dump_name + '.json'
    
    file_opened = read_json(path)
    
    return file_opened

# Acessing API

In [15]:
def get_board_by_name(board_name):
    boards_url = f'https://api.trello.com/1/search?query={board_name}'
    board = get_data_from_trello_api(boards_url)
    
    return board

In [16]:
def get_lists_by_board(board_name, board_id, current_timestamp):
    lists_url = 'https://api.trello.com/1/boards/{0}/lists'
    lists = get_data_from_trello_api(lists_url.format(board_id))
          
    return lists

In [17]:
def get_custom_fields_by_board(board_name, board_id, current_timestamp):
    custom_fields_url = 'https://api.trello.com/1/boards/{0}/customFields'
    
    custom_fields = get_data_from_trello_api(custom_fields_url.format(board_id))
    
    return custom_fields

In [18]:
def get_cards_by_board(board_name, board_id, current_timestamp):
    cards_on_board_url = 'https://api.trello.com/1/boards/{0}/cards/?customFieldItems=true'
    board_cards = get_data_from_trello_api(cards_on_board_url.format(board_id))
    
    return board_cards

In [19]:
def get_members_by_board(board_name, board_id, current_timestamp):
    members_on_board_url = 'https://api.trello.com/1/boards/{0}/members'
    board_members = get_data_from_trello_api(members_on_board_url.format(board_id))
    
    return board_members

# Creating dumps

In [20]:
def create_boards_dump(board_name, current_timestamp):
    boards = get_board_by_name(board_name)

    name_of_dump = f'dumps/{board_name}/{current_timestamp}/dump_board.json'
        
    write_json(name_of_dump, boards)
    
    return name_of_dump

In [21]:
def get_id_board_from_dump(board_name, current_timestamp):
    board = get_board_by_name_from_dump(board_name, current_timestamp)
    id_board = board['boards'][0]['id']

    return id_board

In [22]:
def create_lists_dump(board_name, board_id, current_timestamp):
    lists = get_lists_by_board(board_name, board_id, current_timestamp)
    
    name_of_dump = f'dumps/{board_name}/{current_timestamp}/dump_lists.json'
    write_json(name_of_dump, lists)
        
    return name_of_dump

In [23]:
def create_custom_fields_dump(board_name, board_id, current_timestamp):
    custom_fields = get_custom_fields_by_board(board_name, board_id, current_timestamp)
        
    name_of_dump = f'dumps/{board_name}/{current_timestamp}/dump_custom_field.json'
    write_json(name_of_dump, custom_fields)
        
    return name_of_dump

In [24]:
def create_cards_dump(board_name, board_id, current_timestamp):
    board_cards = get_cards_by_board(board_name, board_id, current_timestamp)    
       
    name_of_dump = f'dumps/{board_name}/{current_timestamp}/dump_cards.json'
    
    write_json(name_of_dump, board_cards)
            
    return name_of_dump

In [25]:
def create_members_dump(board_name, board_id, current_timestamp):
    board_members = get_members_by_board(board_name, board_id, current_timestamp)    
       
    name_of_dump = f'dumps/{board_name}/{current_timestamp}/dump_members.json'
    
    write_json(name_of_dump, board_members)
            
    return name_of_dump

In [26]:
def create_dumps_by_name(board_name, current_timestamp):
    create_folder_for_dumping(board_name, current_timestamp)
    
    create_boards_dump(board_name, current_timestamp)
    
    board_id = get_id_board_from_dump(board_name, current_timestamp)
    
    create_lists_dump(board_name, board_id, current_timestamp)
    create_custom_fields_dump(board_name, board_id, current_timestamp)
    create_cards_dump(board_name, board_id, current_timestamp)
    create_members_dump(board_name, board_id, current_timestamp)

# Getting data from dumps

In [27]:
def get_board_by_name_from_dump(board_name, timestamp):
    board = get_data_from_dump(board_name, 'board', timestamp)
    
    return board

In [28]:
def mapping_lists_by_board_from_dump(board_name, timestamp):
    lists_json = get_data_from_dump(board_name, 'lists', timestamp)
    
    list_map = {}
    for list in lists_json:
        list_map[list['id']] = list['name']
    
    return list_map

In [29]:
def mapping_custom_fields_by_board_from_dump(board_name, timestamp):
    custom_fields_json = get_data_from_dump(board_name, 'custom_field', timestamp)
    
    custom_field_map = {}
    for custom_field in custom_fields_json:
        custom_field_map[custom_field['id']] = custom_field['name']
        
        if custom_field['type'] == 'list':
            options = custom_field['options']
            for option in options:
                custom_field_map[option['id']] = option['value']['text']
    
    return custom_field_map

In [30]:
def mapping_members_by_board_from_dump(board_name, timestamp):
    members_json = get_data_from_dump(board_name, 'members', timestamp)
    
    members_map = {}
    for member in members_json:
        members_map[member['id']] = member['fullName']
    
    return members_map

In [31]:
def get_useful_cards_by_board(board_name, timestamp, custom_field_required):
    cards_raw = get_data_from_dump(board_name, 'cards' , timestamp)
        
    fields = ('id', 'name', 'idList', 'shortUrl', 'customFieldItems', 'idMembers')

    cards = [{key : value for key, value in card.items() if key in fields} for card in cards_raw ]

    custom_fields_map = mapping_custom_fields_by_board_from_dump(board_name, timestamp)

    lists_map = mapping_lists_by_board_from_dump(board_name, timestamp)

    members_map = mapping_members_by_board_from_dump(board_name, timestamp)

    useful_cards = []
    for card in cards:
        
        idListName = lists_map[card['idList']]
        
        if idListName in ['Done']:
            normalized_card = {}

            normalized_card['name'] = card['name']
            normalized_card['shortUrl'] = card['shortUrl']
            normalized_card['idList'] = lists_map[card['idList']]
            
            members_in_card = []
            for member in card['idMembers']:
                members_in_card.append(members_map[member])
            normalized_card['idMember'] = members_in_card
            
            for custom_field in card['customFieldItems']:
                name = custom_fields_map[custom_field['idCustomField']]

                if name in custom_field_required:
                    
                    if 'idValue' in custom_field:
                        name_value = custom_fields_map[custom_field['idValue']]
                        normalized_card[name] = name_value
                        
                    elif 'value' in custom_field:
                        for key, value in custom_field['value'].items():
                            result = value 
                        normalized_card[name] = result                           
                       
            if len(normalized_card) < 3:
                raise Exception(
                    'Make sure all dates are filled in the card: Start, EndDev and End for {0}'.format(card['name']))

            useful_cards.append(normalized_card)
            
    return useful_cards

In [32]:
def create_dataframe_from_trello(board_name, timestamp, custom_field_required):
    cards = get_useful_cards_by_board(board_name, timestamp, custom_field_required)
    df = pd.DataFrame.from_dict(cards)

    df['dev_duration'] = (pd.to_datetime(df['EndDev']).dt.date - pd.to_datetime(df['Start']).dt.date).dt.days
    df['duration'] = (pd.to_datetime(df['End']).dt.date - pd.to_datetime(df['Start']).dt.date).dt.days
    
    df['busday_dev_duration'] = np.busday_count(
        pd.to_datetime(df['Start']).dt.date,
        pd.to_datetime(df['EndDev']).dt.date)
    
    df['busday_duration'] = np.busday_count(
        pd.to_datetime(df['Start']).dt.date,
        pd.to_datetime(df['End']).dt.date)
    
    df['quarter'] = pd.PeriodIndex(df['End'], freq='Q')
    
    df['month'] = pd.PeriodIndex(df['End'], freq='M')

    df["count"] = 1
    
    return df

In [47]:
pd.set_option('display.max_rows', None)

current_timestamp = generate_timestamp()
create_dumps_by_name('CBN', current_timestamp)
custom_field_required = read_json('custom_fields_required.json')
print(custom_field_required)
df = create_dataframe_from_trello('CBN', current_timestamp, custom_field_required)

['Start', 'End', 'EndDev', 'Size/Complexity', 'Impacted parsers', 'New parser', 'Default', 'Priority', 'Date requested (by Google)', 'Commit Date (by Google)']


In [48]:
df

,name,shortUrl,idList,idMember,Commit Date (by Google),Date requested (by Google),Impacted parsers,New parser,Size/Complexity,EndDev,...,End,Start,Default,dev_duration,duration,busday_dev_duration,busday_duration,quarter,month,count
0,GCP_CLOUDAUDIT_b193815534_VPC_SC,https://trello.com/c/FV7nko3g,Done,[diegogr_cit],2021-07-23T15:00:00.000Z,2021-07-16T15:00:00.000Z,Default,No,S,2021-08-06T19:11:36.000Z,...,2021-08-06T20:35:04.000Z,2021-08-05T20:30:09.000Z,NaN,1,1,1,1,2021Q3,2021-08,1
1,SETINEL_EDR_client_migrations,https://trello.com/c/eIZEYUFv,Done,"[diegogr_cit, fmendonca_cit]",NaN,NaN,Customer,No,M,2021-08-03T14:57:12.000Z,...,2021-08-06T18:59:32.000Z,2021-07-27T15:24:00.000Z,NaN,7,10,5,8,2021Q3,2021-08,1
2,SENTINEL_EDR_b183803789,https://trello.com/c/FTPfBGbO,Done,"[fmendonca_cit, diegogr_cit]",2021-06-29T15:00:00.000Z,2021-03-29T15:00:00.000Z,NaN,NaN,S,2021-07-23T21:11:57.000Z,...,2021-07-30T14:39:15.000Z,2021-07-20T19:10:02.000Z,NaN,3,10,3,8,2021Q3,2021-07,1
3,GMAIL_LOGS_b183112646,https://trello.com/c/YRoKpNIX,Done,"[DIEGO DE OLIVEIRA MARANHAO, diegogr_cit]",2021-06-01T15:00:00.000Z,2021-05-18T15:00:00.000Z,NaN,NaN,NaN,2021-07-23T21:57:20.000Z,...,2021-07-28T16:42:00.000Z,2021-07-20T13:37:13.000Z,NaN,3,8,3,6,2021Q3,2021-07,1
4,PAN_FIREWALL,https://trello.com/c/FJ7ZoG5C,Done,"[felipegc_cit, DIEGO DE OLIVEIRA MARANHAO]",2021-07-31T15:00:00.000Z,2021-06-21T15:00:00.000Z,Default,No,XS,2021-07-26T20:31:38.000Z,...,2021-07-27T13:38:45.000Z,2021-07-26T11:16:47.000Z,true,0,1,0,1,2021Q3,2021-07,1
5,BITDEFENDER_ANALYTICS,https://trello.com/c/3uVrURnK,Done,"[diegogr_cit, fmendonca_cit]",2021-05-29T15:00:00.000Z,2021-04-21T15:00:00.000Z,Default,No,M,2021-07-21T14:30:25.000Z,...,2021-07-21T14:39:35.000Z,2021-07-01T13:00:00.000Z,NaN,20,20,14,14,2021Q3,2021-07,1
6,GCP_COMPUTE_CONTEXT_b191179184,https://trello.com/c/sDQ5SLTg,Done,"[DIEGO DE OLIVEIRA MARANHAO, diegogr_cit]",2021-06-18T15:00:00.000Z,2021-06-15T15:00:00.000Z,NaN,NaN,NaN,2021-07-07T22:38:07.000Z,...,2021-07-16T21:16:52.000Z,2021-07-07T13:17:39.000Z,NaN,0,9,0,7,2021Q3,2021-07,1
7,GCP_COMPUTED_CONTEXT (fix),https://trello.com/c/y5RtGsK8,Done,[diegogr_cit],NaN,NaN,Default,No,XS,2021-07-14T14:44:06.000Z,...,2021-07-16T14:41:09.000Z,2021-07-13T15:02:25.000Z,NaN,1,3,1,3,2021Q3,2021-07,1
8,CBN - GCP Security Center (add is_alert and is...,https://trello.com/c/OocAPXtk,Done,[felipegc_cit],2021-07-13T14:28:00.000Z,2021-07-12T15:00:00.000Z,Default,No,XS,2021-07-13T15:00:00.000Z,...,2021-07-13T16:00:00.000Z,2021-07-12T14:28:00.000Z,true,1,1,1,1,2021Q3,2021-07,1
9,PROOFPOINT_ON_DEMAND(fromhashed email field to...,https://trello.com/c/KJGHodwG,Done,"[felipegc_cit, DIEGO DE OLIVEIRA MARANHAO]",2021-03-31T15:00:00.000Z,2021-01-08T15:00:00.000Z,Default,No,NaN,2021-06-25T19:40:11.000Z,...,2021-07-13T14:22:50.000Z,2021-05-25T15:01:41.000Z,true,31,49,23,35,2021Q3,2021-07,1


In [ ]:
cancelled = df[(df['idList'] == 'Cancelled')]
done = df[(df['idList'] == 'Done')]

# Calculating General Estimatives

In [ ]:
def get_extremes(data_frame, duration_column):
    upper_q = partial(pd.Series.quantile, q=0.95)
    lower_q = partial(pd.Series.quantile, q=0.05)

    upper_extremes = data_frame[duration_column].agg([upper_q])["quantile"]
    lower_extremes = data_frame[duration_column].agg([lower_q])["quantile"]
    
    return lower_extremes, upper_extremes

In [ ]:
def calculate_estimatives_by_duration_column(data_frame, duration_column, print_results=True):
    lower_extremes, upper_extremes = get_extremes(data_frame, duration_column)
    
    done_extremes_removed = data_frame[(data_frame[duration_column] > lower_extremes) & (data_frame[duration_column] < upper_extremes)]
    mean_removed_extremes = done_extremes_removed[duration_column].mean()
    
    small_q = partial(pd.Series.quantile, q=0.25)
    small_limit = done_extremes_removed[duration_column].agg([small_q])["quantile"]
    
    small_extremes_removed = done_extremes_removed[(done_extremes_removed[duration_column] <= small_limit)]
    not_small_extremes_removed = done_extremes_removed[(done_extremes_removed[duration_column] > small_limit)]
    
    mean_small_extremes_removed = small_extremes_removed[duration_column].mean()
    mean_not_small_extremes_removed = not_small_extremes_removed[duration_column].mean()
    
    total_developed = len(data_frame)
    
    if print_results:
        features = ('lower_extremes', 'upper_extremes', 'small limit', 'Done estimate (with "extremes" removed)',
                   'Done estimate for "Small" ones', 'Done estimate for "Big" ones', 'Total_developed')
        values = (lower_extremes, upper_extremes, small_limit, mean_removed_extremes, mean_small_extremes_removed, 
                 mean_not_small_extremes_removed, total_developed)
        general_estimatives = {'Feature':features, 'Value':values}
        general_estimatives_df = pd.DataFrame(data=general_estimatives)
        display(general_estimatives_df)
    
    return done_extremes_removed

In [ ]:
def calculate_estimatives(data_frame):
    display(Markdown('### Total Duration:'))
    calculate_estimatives_by_duration_column(data_frame, 'duration')
    print('\n')
    display(Markdown('### Total Dev Duration:'))
    calculate_estimatives_by_duration_column(data_frame, 'dev_duration')
    print('\n')
    display(Markdown('## BUSINESS DAY'))
    print('\n')
    display(Markdown('### Business Day Duration:'))
    calculate_estimatives_by_duration_column(data_frame, 'busday_duration')
    print('\n')
    display(Markdown('### Business Day Dev Duration:'))
    calculate_estimatives_by_duration_column(data_frame, 'busday_dev_duration')

In [ ]:
def generate_table_amount_delivered_by_period(df, total_fte, period): # quarter or month
    by_period = df[[period, 'count']].groupby(period).sum('count')

    by_period_fte = pd.merge(by_period, total_fte, on=period, how='left')

    period_result = by_period_fte[[period, 'count', 'fte']].groupby([period,'count']).sum('fte')
    period_result.reset_index(drop=False, inplace=True)

    period_result['parsers_per_fte'] = period_result['count'].div(period_result['fte'])

    period_result[period] = period_result[period].astype(str)

    return period_result

In [ ]:
def generate_chart_amount_delivered_by_period(df, period):
    ax = df[[period,'fte', 'parsers_per_fte']].plot(x=period, linestyle='-', marker='o', color=['orange', 'pink'])
    df[[period,'count']].plot(x=period, kind='bar', ax=ax)
    display(df)

In [ ]:
def generate_chart_and_table_amount_delivered_by_period(df, total_fte, period):
    if period in ('month', 'quarter'):
        period_result = generate_table_amount_delivered_by_period(df, total_fte, period)
        generate_chart_amount_delivered_by_period(period_result, period)
    else:
        raise Exception(f'"{period}" is not defined. Must be "month" or "quarter".')

# General Estimatives

In [ ]:
calculate_estimatives(done)

# Amount delivered by month

In [ ]:
generate_chart_and_table_amount_delivered_by_period(df, total_fte, 'month')

# Amount delivered by quarter

In [ ]:
generate_chart_and_table_amount_delivered_by_period(df, total_fte, 'quarter')